### Setup

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import tqdm
import cobra
from cobra.io import read_sbml_model
from pathlib import Path
from collections import Counter
import os
import seaborn as sns
import numpy as np
from scipy import stats
from sklearn.linear_model import LinearRegression
from scipy.stats import linregress
from scipy.stats import spearmanr

### Functions

In [3373]:
def get_betweenness_centrality_log10(model_graph_df):
    # calculate betweenness centrality
    model_betweenness_centrality = nx.betweenness_centrality(model_graph_df, weight='weight', normalized=False)
    
    # strip whitespace from keys and log-transform the values
    log10_betweenness_centrality = {key.strip(): np.log10(value + 1) for key, value in model_betweenness_centrality.items()}
    
    return log10_betweenness_centrality


def get_bridging_centrality_log10(model_graph_df):
    # calculate betweenness centrality
    betweenness = nx.betweenness_centrality(model_graph_df, weight='weight', normalized=False)

    # calculate the bridging coefficient for each node
    bridging_coefficient = {}
    for node in model_graph_df.nodes():
        
        # for directed graphs, consider successors and predecessors
        if model_graph_df.is_directed():
            successors = list(model_graph_df.successors(node))
            predecessors = list(model_graph_df.predecessors(node))
            degree_sum = sum(model_graph_df.out_degree(successor) for successor in successors) + \
                         sum(model_graph_df.in_degree(predecessor) for predecessor in predecessors)
        else:
            # for undirected graphs, consider neighbors
            neighbors = list(model_graph_df.neighbors(node))
            degree_sum = sum(model_graph_df.degree(neighbor) for neighbor in neighbors)

        bridging_coefficient[node] = 1 / degree_sum if degree_sum > 0 else 0

    # calculate bridging centrality
    bridging_centrality = {node: betweenness[node] * bridging_coefficient[node] for node in model_graph_df.nodes()}

    # strip whitespace from keys and log-transform the values
    log10_bridging_centrality = {key.strip(): np.log10(value + 1) for key, value in bridging_centrality.items()}

    return log10_bridging_centrality

def get_clustering_coefficient(model_graph_df):
    model_clustering_coefficient = nx.clustering(model_graph_df)

    # strip whitespace from keys and log-transform the values
    clustering_coefficient = {key.strip(): value for key, value in model_clustering_coefficient.items()}

    return clustering_coefficient

def get_degree(model_graph_df, log=False, directional=False):
    model_degree = dict(model_graph_df.degree())
    model_degree = {key.strip(): value for key, value in model_degree.items()}
    
    if (directional):
        c=0
    
    # strip whitespace from keys and log-transform the values
    if (log):
        model_degree = {key.strip(): np.log10(value + 1) for key, value in model_degree.items()}
    
    return model_degree

In [3502]:
def dict_isolate(dic, val):
    match_keys=[]
    for key, value in dic.items():
        if (value==val):
            match_keys.append(key)
    return match_keys

def df_isolate(df, col, val):
    return df[df[col]==val]

def log_columns(df, cols, base=2):
    for col in cols:
        if (base==2):
            df[f"log_{col}"]=np.log(df[col]+1)
        elif (base==10):
            df[f"log_{col}"]=np.log10(df[col]+1)
        else:
            print("Choose valid base (2 or 10)")
    return df

from sklearn import preprocessing
def normalize(data, lower, upper):
    data_norm=[lower + (upper - lower) * x for x in data]
    return data_norm

def adj_columns(df, cols, prefix):
    for col in cols:
        d=np.array(df[col])
        df[f"{prefix}_{col}"]=preprocessing.normalize([d]).tolist()[0]
    return df

def z_score(df, cols, prefix):
    for col in cols:
        d=np.array(df[col])
        df[f"{prefix}_{col}"]=stats.zscore(d).tolist()
    return df

def scale(df, cols, lb=0, ub=1):
    for col in cols:
        df[f"scale_{col}"]=scale_list(list(df[col]), lb, ub)
    return df

def scale_number(unscaled, to_min, to_max, from_min, from_max):
    return (to_max-to_min)*(unscaled-from_min)/(from_max-from_min)+to_min

def scale_list(l, to_min, to_max):
    return [scale_number(i, to_min, to_max, min(l), max(l)) for i in l]

In [3503]:
def replace_index(df, replace_pairs):
    for r1, r2 in replace_pairs:
        df.index=df.index.str.replace(r1,r2)
    return df

def replace_column(df, replace_pairs):
    for r1, r2 in replace_pairs:
        df.columns=df.columns.str.replace(r1,r2)
    return df

def sort_dict(d):
    return {k: v for k, v in sorted(d.items(), key=lambda item: item[1])}

def sorted_counter(data):
    return sort_dict(Counter(data))

In [3504]:
# CHANGE BASED ON UPDATES MADE BELOW
# assign bins for histogram
def assign_bins(df, col, bins, return_hist=False):
    col_data=list(df[col])
    # make sure min of data is not zero
    if (min(col_data)==0):
        print("Warning, minimum of data is zero.")
    hist,bin_edges=np.histogram(col_data, bins=bins)
    hist=list(hist)
    bin_edges=list(bin_edges)
    
    # not needed
#     bin_edges.insert(0, 0)
#     max_p1=max(col_data)+1
#     bin_edges.append(max_p1)
    
    # get bin edge pairs
    bin_edge_pairs={}
    for i in range(len(bin_edges)):
        if (i>=len(bin_edges)-1):
            break
        pair=(bin_edges[i], bin_edges[i+1])
        bin_edge_pairs[pair]=f"g{i}"        
        
    # bound the data
    df_hist=pd.DataFrame()
    for bin_pair, label in zip(bin_edge_pairs.keys(), bin_edge_pairs.values()):
        df_bounded=df.loc[df[col].between(bin_pair[0], bin_pair[1])]
        df_bounded["hist_label"]=label
        df_hist=pd.concat([df_hist, df_bounded])
        
    if (return_hist):
        return df_hist, hist, bin_edges, bin_edge_pairs
    else:
        return df_hist
    
# computes fraction of essential reactions
def compute_fraction_essential_hist(df_hist, hist, bin_edges, bin_edge_pairs, compute_ratio=False):
    df_ess=pd.DataFrame()
    df_ess["hist_label"]=list(bin_edge_pairs.values())
    df_ess["hist_counts"]=hist
    ess_counts=[]
    noness_counts=[]
    ess_fracs=[]
    bin_centers=[]
    left_bin=[]
    right_bin=[]
    for pair, label in zip(bin_edge_pairs.keys(), bin_edge_pairs.values()):
        bin_centers.append(np.mean(pair))
        left_bin.append(pair[0])
        right_bin.append(pair[1])
        df_hist_label=df_hist[df_hist["hist_label"]==label]
        num_ess=len(df_hist_label[df_hist_label["ess"]==1])
        num_noness=len(df_hist_label[df_hist_label["ess"]==0])
        if (compute_ratio):
            if (num_noness==0):
                frac_ess=0
            else:
                frac_ess=num_ess/num_noness
        else:
            if ((num_ess+num_noness)==0):
                frac_ess=0
            else:
                frac_ess=num_ess/(num_ess+num_noness)
        
        # append data
        ess_counts.append(num_ess)
        noness_counts.append(num_noness)
        ess_fracs.append(frac_ess)
        
    # update df
    df_ess["num_ess"]=ess_counts
    df_ess["num_noness"]=noness_counts
    df_ess["frac_ess"]=ess_fracs
    df_ess["bin_centers"]=bin_centers
    df_ess["left_bin"]=left_bin
    df_ess["right_bin"]=right_bin
    
    # return 
    return df_ess


In [12]:
# counts reactant and product sharing across all reactions
def all_prods_reacts_dict(fba_ecoli_lofo, ecoli_model, rev=False):
    
    all_reactants=[]
    all_products=[]
    for rxn in list(fba_ecoli_lofo.keys()):
        rxn_df=ecoli_model.reactions.get_by_id(rxn)
        rxn_reactants=rxn_df.reactants
        rxn_products=rxn_df.products
        rxn_reactants=[i.id for i in rxn_reactants]
        rxn_products=[i.id for i in rxn_products]

        rxn_reversibility=rxn_df.reversibility
        if (rev):
            if (rxn_reversibility):
                all_mets=rxn_reactants+rxn_products
                all_reactants+=all_mets
                all_products+=all_mets
            else:
                all_reactants+=rxn_reactants
                all_products+=rxn_products
        else:
            all_reactants+=rxn_reactants
            all_products+=rxn_products
        
    # count and convert to dictionary
    all_products_dict=Counter(all_products)
    all_reactants_dict=Counter(all_reactants)

    # sort dictionaries and create a list of metabolites to ignore because they are too prevelant
    all_products_dict={k: v for k, v in sorted(all_products_dict.items(), key=lambda item: item[1])}
    all_reactants_dict={k: v for k, v in sorted(all_reactants_dict.items(), key=lambda item: item[1])}
        
    return all_reactants_dict, all_products_dict

In [13]:
all_reactants_dict, all_products_dict=all_prods_reacts_dict(fba_ecoli_lofo, ecoli_model, rev=False)

In [14]:
# print(stats.ttest_ind(list(all_reactants_ess_dict.values()), list(all_reactants_noness_dict.values()), equal_var=False))
# print(stats.ttest_ind(list(all_products_ess_dict.values()), list(all_products_noness_dict.values()), equal_var=False))

In [15]:
# count and convert to dictionary
# all_products_dict=Counter(all_products)
# all_reactants_dict=Counter(all_reactants)

# # sort dictionaries and create a list of metabolites to ignore because they are too prevelant
# all_products_dict={k: v for k, v in sorted(all_products_dict.items(), key=lambda item: item[1])}
# all_reactants_dict={k: v for k, v in sorted(all_reactants_dict.items(), key=lambda item: item[1])}

In [1094]:
# create dataframe info in one dict
def split_react_prod_dict_by_ess(fba_ecoli_lofo):
    all_reactants_ess=[]
    all_products_ess=[]
    all_reactants_noness=[]
    all_products_noness=[]

    for rxn, obj in zip(fba_ecoli_lofo.keys(), fba_ecoli_lofo.values()):
        rxn_df=ecoli_model.reactions.get_by_id(rxn)
        rxn_reactants=rxn_df.reactants
        rxn_products=rxn_df.products
        rxn_reactants=[i.id for i in rxn_reactants]
        rxn_products=[i.id for i in rxn_products]
        if (obj < 2):
            all_reactants_ess+=rxn_reactants
            all_products_ess+=rxn_products
        else:
            all_reactants_noness+=rxn_reactants
            all_products_noness+=rxn_products

    # create dicts
    all_reactants_ess_dict=dict(Counter(all_reactants_ess))
    all_products_ess_dict=dict(Counter(all_products_ess))
    all_reactants_noness_dict=dict(Counter(all_reactants_noness))
    all_products_noness_dict=dict(Counter(all_products_noness))
    
    return all_reactants_ess_dict, all_products_ess_dict, all_reactants_noness_dict, all_products_noness_dict

In [3]:
from centrality_functions.py import 

ModuleNotFoundError: No module named 'centrality_functions.py'; 'centrality_functions' is not a package